# **Deploying models registered in Azure ML**

Once a model is registered in Azure ML, we can deploy them using either the UI interface in Azure ML Studio, the Azure ML CLI v2 from a console, or the azureml-mlflow plugin for MLflow. Use the approach it best suites your needs. Here we will demostrate how to do that using the MLflow deployment plugin.

## Deploying models registered in Azure ML to Managed Inference

To make the deployment happen, we will need a deployment client. Deployments can be generated using both the Python API for MLflow or MLflow CLI. In both cases, a JSON configuration file needs to be indicated with the details of the deployment you want to achieve.

Run following cells to Configure the variables

In [ ]:
aml_region = ""
subscription_id = ""
aml_resource_group = ""
aml_workspace_name = ""

In [ ]:
azureml_mlflow_uri=f"azureml://{aml_region}.api.azureml.ms/mlflow/v1.0/subscriptions/{subscription_id}/resourceGroups/{aml_resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{aml_workspace_name}"
print(azureml_mlflow_uri)

In [ ]:
import mlflow
# Indicate to MLflow from where the models need to be pulled from. Currently, the source and target URLs need to be the same.
mlflow.set_tracking_uri(azureml_mlflow_uri)

In [ ]:
subscription_id = ""
resource_group = ""                   
workspace_name = "" 

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()
#Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

## Deploying the registered model

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model
from azure.ai.ml.constants import AssetTypes

model_name = 'databricks-diabetes-prediction'
model_uri = "models:/databricks-diabetes-prediction/1"

## Download the model locally

The following downloads the model from your Databricks workspace to your local machine.

In [ ]:
import os
try:
    os.mkdir(model_name)
except:
    None
model_path = mlflow.tracking.artifact_utils._download_artifact_from_uri(artifact_uri=model_uri, output_path=model_name)

## Load local MLFlow model
In the following, the Model class is used to package the model for use with Docker and later for deployment as a real-time endpoint.

In [ ]:
mlflow_model = Model(
    path=model_path,
    type=AssetTypes.MLFLOW_MODEL,
    name=model_name,
    description="Model deployed with V2"
)

## Create an Azure ML managed online endpoint

After we load the model, we can create a managed online endpoint for deployment.

In [ ]:
online_endpoint_name = "DP-endpoint"
 
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Diabetes prediction endpoint",
    #auth_mode="key",
    tags={"model": model_name},
)


In [ ]:
ml_client.begin_create_or_update(endpoint)

## Deploy registered model to managed online endpoint

Now that we have ve created an endpoint, we can create a deployment on it. Azure Machine Learning manage online endpoints can have one or multiple deployments and traffic can be configured for each deployment. In this example, we create only one deployment to serve all the traffic, named sklearn-deployment.

In [ ]:
sklearn_deployment = ManagedOnlineDeployment(
                      name="sklearn-deployment",
                      endpoint_name=online_endpoint_name,
                      model=mlflow_model,
                      instance_type="Standard_DS2_v2",
                      instance_count=1,
                  )

In [ ]:
ml_client.begin_create_or_update(sklearn_deployment)
endpoint.traffic = {"sklearn-deployment": 100}
ml_client.begin_create_or_update(endpoint)

```
Got this error while deployed the registered model
```
ImportError: cannot import name 'parse_json_input' from 'mlflow.pyfunc.scoring_server' (/opt/miniconda/envs/userenv/lib/python3.8/site-packages/mlflow/pyfunc/scoring_server/__init__.py). Didn't find a solution I think this is a bug which needs to be resolved!

found another tutorial[https://learn.microsoft.com/en-us/azure/databricks/_static/notebooks/mlflow/mlflow-quick-start-deployment-azure.html], does the same job but this also got the same error!